# Build a Multi-Agent Using AutoGen

# Packages 설치

In [1]:
# Install packages
%pip install --upgrade pip

%pip install pyautogen
%pip install openai
%pip install chroma             # Chroma DB
%pip install python-dotenv
%pip install markdownify
%pip install apify_client
%pip install duckduckgo-search  # DuckDuckGo Search
%pip install autogen_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 초기화 

## .env 파일 편집 
 - 아래 3가지 API 키를 .env 파일에 업데이트 합니다.
    - OPENAI_API_KEY=<YOUR_API_KEY>
    - BING_API_KEY=<YOUR_BING_API_KEY>
    - APIFY_API_KEY=<YOUR_APIFY_API_KEY>

In [2]:
import os
import autogen
from dotenv import load_dotenv

load_dotenv()
# Save your OPENAI_API_KEY in a .env file in the same directory as this script
llm_config_dict = {"config_list": [{"model": "gpt-4o", "api_type": "azure", "api_key": os.environ["OPENAI_API_KEY"],"base_url": os.environ["OPENAI_URL"], "api_version": "2024-02-01"}]}

In [3]:
# %pip install "flaml[autogen]"   # AutoML

In [4]:
# print(llm_config_dict["config_list"][0])

# Two-agent chat: 
 - 가장 간단한 대화 패턴으로, 두 에이전트가 Autogen을 사용하여 서로 대화
 - summary_method: 채팅에서 요약을 얻기 위한 방법으로 채팅 기록에서 마지막 메시지를 요약으로 사용하는 last_msg와, 채팅 기록을 반영하여 LLM 호출을 사용해 요약을 제공하는 reflection_with_llm 등이 가능

<img src="./images/autogen_image1.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [5]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn. You ask meaningful and precise follow up questions and are eager to learn more.",
    llm_config=llm_config_dict,
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are an expert at Large Language model research and you teach cutting edge AI development technologies.",
    llm_config=llm_config_dict,
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    # message="How to prove or test that Artificial Intelligence is actually intelligent?",
    message="인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?",
    summary_method="reflection_with_llm", # to summarize the body of conversation
    max_turns=2,
)

Student_Agent (to Teacher_Agent):

인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

인공지능이 실제로 지능적임을 증명하거나 테스트하는 것은 복잡하고 논란이 많은 주제입니다. 전통적으로, 이는 다양한 접근 방법을 통해 평가될 수 있습니다. 몇 가지 주요 방법론을 소개하겠습니다.

1. **튜링 테스트**: 알란 튜링이 제안한 테스트로, 기계가 인간과 구별할 수 없을 정도로 자연스러운 대화를 할 수 있다면 지능적이라고 판단합니다. 그러나 이 테스트는 인간과 유사한 대화 능력만을 측정하기 때문에 지능의 포괄적인 측면을 다루지 못합니다.

2. **CAPTCHA**: 사람이 쉽게 풀 수 있지만 기계는 어렵게 여기는 문제를 통해 인공지능의 한계를 식별하는 방법입니다. 이는 인간이 기계보다 우월한 특정 인지 능력을 활용해 AI의 지능 수준을 평가합니다.

3. **지능 지수 테스트(Artificial IQ Tests)**: 인간의 IQ 테스트를 변형하여 기계의 문제 해결 능력, 논리적 추론, 학습 능력 등을 평가합니다. 그러나 이러한 방법은 인간 중심적 지능 모형에 기반하고 있어, AI에 적합하지 않을 수 있습니다.

4. **공동 작업 및 적응 능력 평가**: AI가 새로운 상황에 적응하거나 인간과 효율적으로 협력할 수 있는 능력을 평가합니다. 이는 AI가 정해진 룰을 넘어 창의적이거나 유연한 방식

요약은 initiate_chat 메서드에 의해 반환된 ChatResult 타입의 chat_result 객체에 포함

In [6]:
print(chat_result.summary)

The conversation discusses how AI's intelligence can be evaluated, focusing on traditional methods like the Turing Test and benchmark tests, as well as newer approaches to assess understanding, emotion recognition, and ethical decision-making. It highlights the complexity of proving AI's genuine understanding or consciousness and the ongoing research in areas like emotion analysis, ethical simulations, and large-scale common-sense testing. Despite technological advances, the question of AI's true understanding or consciousness remains open and contentious, requiring further scientific and philosophical exploration.


위 예시에서, 요약 방법은 `reflection_with_llm`으로 설정되어 있으며, 이는 LLM을 호출하여 대화 메시지 목록을 처리하고 요약합니다.

처음에 요약 방법은 수신자의 LLM을 사용하려 시도하지만, 사용 불가능할 경우 발신자의 LLM을 기본으로 사용합니다.

여기서 수신자는 “Teacher_Agent”이고, 발신자는 “Student_Agent”가 됩니다.

LLM의 입력 프롬프트는 다음과 같습니다.

In [7]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


In [8]:
import json
from pprint import pprint

print(json.dumps(chat_result.cost, indent=4))
pprint(chat_result.chat_history)

{
    "usage_including_cached_inference": {
        "total_cost": 0.017875,
        "gpt-4o-2024-08-06": {
            "cost": 0.017875,
            "prompt_tokens": 2398,
            "completion_tokens": 1188,
            "total_tokens": 3586
        }
    },
    "usage_excluding_cached_inference": {
        "total_cost": 0
    }
}
[{'content': '인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': '인공지능이 실제로 지능적임을 증명하거나 테스트하는 것은 복잡하고 논란이 많은 주제입니다. 전통적으로, 이는 다양한 '
             '접근 방법을 통해 평가될 수 있습니다. 몇 가지 주요 방법론을 소개하겠습니다.\n'
             '\n'
             '1. **튜링 테스트**: 알란 튜링이 제안한 테스트로, 기계가 인간과 구별할 수 없을 정도로 자연스러운 대화를 할 '
             '수 있다면 지능적이라고 판단합니다. 그러나 이 테스트는 인간과 유사한 대화 능력만을 측정하기 때문에 지능의 포괄적인 '
             '측면을 다루지 못합니다.\n'
             '\n'
             '2. **CAPTCHA**: 사람이 쉽게 풀 수 있지만 기계는 어렵게 여기는 문제를 통해 인공지능의 한계를 식별하는 '
             '방법입니다. 이는 인간이 기계보다 우월한 특정 인지 능력을 활용해 AI의 지능 수준을 평가합니다.\n'
             '\n'
             '3. *

# Sequential chat: 
 - 두 에이전트 간의 일련의 대화로, 이전 대화의 요약을 다음 대화의 맥락으로 전달하는 연속성 메커니즘 (Carryover Mechanism)에 의해 연결

<img src="./images/autogen_image2.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [9]:
from autogen import ConversableAgent

report_agent = ConversableAgent(
    name="Report_Agent",
    system_message="You are responsible for creating a report by extracting insights from the chat history.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Researcher Agent discovers the potential of AGI.
researcher_agent = ConversableAgent(
    name="Researcher_Agent",
    system_message="You explore and describe the potential capabilities and advancements of AGI.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Ethicist Agent evaluates the ethical implications of AGI.
ethicist_agent = ConversableAgent(
    name="Ethicist_Agent",
    system_message="You evaluate the ethical implications of AGI based on the research findings.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Economist Agent analyzes the economic impact of AGI.
economist_agent = ConversableAgent(
    name="Economist_Agent",
    system_message="You analyze the economic impact of AGI based on the ethical evaluations.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Policy Maker Agent develops policies based on the findings of the previous agents.
policy_maker_agent = ConversableAgent(
    name="Policy_Maker_Agent",
    system_message="You develop policies to manage AGI based on the economic analysis.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)


In [10]:
# Here we start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.

chat_results = report_agent.initiate_chats(
    [
        {
            "recipient": researcher_agent,
            "message": "Discover the potential of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": ethicist_agent,
            "message": "Evaluate the ethical implications of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": economist_agent,
            "message": "Analyze the economic impact of AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": policy_maker_agent,
            "message": "Develop policies to manage AGI",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
Report_Agent (to Researcher_Agent):

Discover the potential of AGI

--------------------------------------------------------------------------------
Researcher_Agent (to Report_Agent):

Artificial General Intelligence (AGI) represents a level of machine intelligence that simulates human-like cognitive abilities. Unlike narrow AI systems, which excel at specific tasks, AGI is envisioned to perform any intellectual task a human can do, and possibly even more. Here’s an exploration of the potential capabilities and advancements of AGI:

### 1. **Cognitive Skills and Problem-Solving:**
   - **Cross-Domain Learning:** AGI can learn and transfer knowledge across different fields without being explicitly programmed for each domain. This implies solving a wide variety of problems using insights from vastly di

In [11]:
print("First Chat Summary: ", chat_results[0].summary)
print("\n******************************************************************************************\n")
print("Second Chat Summary: ", chat_results[1].summary)
print("\n******************************************************************************************\n")
print("Third Chat Summary: ", chat_results[2].summary)
print("\n******************************************************************************************\n")
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  Artificial General Intelligence (AGI) represents a level of machine intelligence that simulates human-like cognitive abilities. Unlike narrow AI systems, which excel at specific tasks, AGI is envisioned to perform any intellectual task a human can do, and possibly even more. Here’s an exploration of the potential capabilities and advancements of AGI:

### 1. **Cognitive Skills and Problem-Solving:**
   - **Cross-Domain Learning:** AGI can learn and transfer knowledge across different fields without being explicitly programmed for each domain. This implies solving a wide variety of problems using insights from vastly different subjects.
   - **Complex Problem Solving:** With advanced reasoning abilities, AGI could tackle complex and multidisciplinary problems such as climate change, global health issues, and financial systems analysis.

### 2. **Adaptability and Learning:**
   - **Continuous Learning:** Unlike current AI models that require vast amounts of labeled d

# 도구 (Tool) 사용
이 섹션에서는 에이전트 시스템의 도구 사용 기능을 시연합니다. Web Search와 RAG는 사람들이 매일 사용하는 인기 있는 기능으로, AutoGen을 사용하여 다음 작업을 수행하는 방법을 알아봅니다:

* Bing API를 사용한 웹 검색 
* Apify API를 사용한 웹 스크래핑
* RAG을 통한 질문과 답변

자세한 정보: https://microsoft.github.io/autogen/docs/tutorial/tool-use 

## Bing API를 사용한 웹 검색

In [12]:
# Put your Bing API key in the .env file in the same directory as this script
BING_API_KEY = os.environ["BING_API_KEY"]

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

In [13]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": BING_API_KEY},
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

In [14]:
# What is Artificial General Intelligence ?

task1 = """
최신 에이전트 기술을 알아보고 싶어요.
"""

user_proxy.initiate_chat(web_surfer, message=task1, max_turns=1)

user_proxy (to web_surfer):


최신 에이전트 기술을 알아보고 싶어요.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: 최신 에이전트 기술 2025
Title: 최신 에이전트 기술 2025 - Search
Viewport position: Showing page 1 of 1.
A Bing search for '최신 에이전트 기술 2025' found 10 results:

## Web Results
1. [2025년을 주도할 기술 트렌드: 에이전트형 AI](https://jazzpocket.tistory.com/entry/2025%EB%85%84%EC%9D%84-%EC%A3%BC%EB%8F%84%ED%95%A0-%EA%B8%B0%EC%88%A0-%ED%8A%B8%EB%A0%8C%EB%93%9C-%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8%ED%98%95-AI)
2025년 IT 업계를 이끌 주요 기술 중 하나로 에이전트형 AI(Agentic AI)가 선정되었습니다.이는 단순히 명령에 반응하는 시스템에서 벗어나, 자율적으로 의사결정을 내리고 복잡한 작업을 수행할 수 있는 인공지능 기술을 의미합니다. 에이전트형 AI는 제조, 금융, 의료 등 다양한 산업에서 혁신을 이끌며 ...

2. [가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 등 선정](https://www.aitimes.com/news/articleView.html?idxno=164495)
글로벌 시장조사 기관 가트너가 2025년 주요 기술 트렌드로 인공지능 (AI) 에이전트를 가장 먼저 꼽았다. 여기에 최

ChatResult(chat_id=None, chat_history=[{'content': '\n최신 에이전트 기술을 알아보고 싶어요.\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Address: bing: 최신 에이전트 기술 2025\nTitle: 최신 에이전트 기술 2025 - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for \'최신 에이전트 기술 2025\' found 10 results:\n\n## Web Results\n1. [2025년을 주도할 기술 트렌드: 에이전트형 AI](https://jazzpocket.tistory.com/entry/2025%EB%85%84%EC%9D%84-%EC%A3%BC%EB%8F%84%ED%95%A0-%EA%B8%B0%EC%88%A0-%ED%8A%B8%EB%A0%8C%EB%93%9C-%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8%ED%98%95-AI)\n2025년 IT 업계를 이끌 주요 기술 중 하나로 에이전트형 AI(Agentic AI)가 선정되었습니다.이는 단순히 명령에 반응하는 시스템에서 벗어나, 자율적으로 의사결정을 내리고 복잡한 작업을 수행할 수 있는 인공지능 기술을 의미합니다. 에이전트형 AI는 제조, 금융, 의료 등 다양한 산업에서 혁신을 이끌며 ...\n\n2. [가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 등 선정](https://www.aitimes.com/news/articleView.html?idxno=164495)\n글로벌 시장조사 기관 가트너가 2025년 주요 기술 트렌드로 인공지능 (AI) 에이전트를 가장 먼저 꼽았다. 여기에 최근 떠오르는 \'공간 지능\' 등 AI 관련 기술이 무려 9개 항목을 차지했다. 가트너는 21일 (현지시간) \'2025년 10대 전략 기

In [15]:
task2 = "Click on the first link."
user_proxy.initiate_chat(web_surfer, message=task2)

user_proxy (to web_surfer):

Click on the first link.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION page_down...
web_surfer (to user_proxy):

Address: bing: 최신 에이전트 기술 2025
Title: 최신 에이전트 기술 2025 - Search
Viewport position: Showing page 1 of 1.
A Bing search for '최신 에이전트 기술 2025' found 10 results:

## Web Results
1. [2025년을 주도할 기술 트렌드: 에이전트형 AI](https://jazzpocket.tistory.com/entry/2025%EB%85%84%EC%9D%84-%EC%A3%BC%EB%8F%84%ED%95%A0-%EA%B8%B0%EC%88%A0-%ED%8A%B8%EB%A0%8C%EB%93%9C-%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8%ED%98%95-AI)
2025년 IT 업계를 이끌 주요 기술 중 하나로 에이전트형 AI(Agentic AI)가 선정되었습니다.이는 단순히 명령에 반응하는 시스템에서 벗어나, 자율적으로 의사결정을 내리고 복잡한 작업을 수행할 수 있는 인공지능 기술을 의미합니다. 에이전트형 AI는 제조, 금융, 의료 등 다양한 산업에서 혁신을 이끌며 ...

2. [가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 등 선정](https://www.aitimes.com/news/articleView.html?idxno=164495)
글로벌 시장조사 기관 가트너가 2025년 주요 기술 트렌드로 인공지능 (AI) 에이전트를 가장 먼저 꼽았다. 여기에 최근 떠오르는 '공간 지능'

ChatResult(chat_id=None, chat_history=[{'content': 'Click on the first link.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Address: bing: 최신 에이전트 기술 2025\nTitle: 최신 에이전트 기술 2025 - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for \'최신 에이전트 기술 2025\' found 10 results:\n\n## Web Results\n1. [2025년을 주도할 기술 트렌드: 에이전트형 AI](https://jazzpocket.tistory.com/entry/2025%EB%85%84%EC%9D%84-%EC%A3%BC%EB%8F%84%ED%95%A0-%EA%B8%B0%EC%88%A0-%ED%8A%B8%EB%A0%8C%EB%93%9C-%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8%ED%98%95-AI)\n2025년 IT 업계를 이끌 주요 기술 중 하나로 에이전트형 AI(Agentic AI)가 선정되었습니다.이는 단순히 명령에 반응하는 시스템에서 벗어나, 자율적으로 의사결정을 내리고 복잡한 작업을 수행할 수 있는 인공지능 기술을 의미합니다. 에이전트형 AI는 제조, 금융, 의료 등 다양한 산업에서 혁신을 이끌며 ...\n\n2. [가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 등 선정](https://www.aitimes.com/news/articleView.html?idxno=164495)\n글로벌 시장조사 기관 가트너가 2025년 주요 기술 트렌드로 인공지능 (AI) 에이전트를 가장 먼저 꼽았다. 여기에 최근 떠오르는 \'공간 지능\' 등 AI 관련 기술이 무려 9개 항목을 차지했다. 가트너는 21일 (현지시간) \'2025년 10대 전략 기술

In [16]:
task3 = "What is the difference beween AGI and classic AI?"
user_proxy.initiate_chat(web_surfer, message=task3)

user_proxy (to web_surfer):

What is the difference beween AGI and classic AI?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: difference between AGI and classic AI
Title: difference between AGI and classic AI - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'difference between AGI and classic AI' found 10 results:

## Web Results
1. [Difference between AGI vs AI - GeeksforGeeks](https://www.geeksforgeeks.org/difference-between-agi-vs-ai/)
AGI refers to systems capable of understanding, learning and performing any intellectual task at human-level capability. AI refers to systems designed to perform specific tasks or solve particular problems efficiently. AGI referred to as strong AI or general AI. AI referred to as narrow AI or weak AI.

2. [There's AI, and Then There's AGI: What You Need to Know to Tell .

ChatResult(chat_id=None, chat_history=[{'content': 'What is the difference beween AGI and classic AI?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Address: bing: difference between AGI and classic AI\nTitle: difference between AGI and classic AI - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for 'difference between AGI and classic AI' found 10 results:\n\n## Web Results\n1. [Difference between AGI vs AI - GeeksforGeeks](https://www.geeksforgeeks.org/difference-between-agi-vs-ai/)\nAGI refers to systems capable of understanding, learning and performing any intellectual task at human-level capability. AI refers to systems designed to perform specific tasks or solve particular problems efficiently. AGI referred to as strong AI or general AI. AI referred to as narrow AI or weak AI.\n\n2. [There's AI, and Then There's AGI: What You Need to Know to Tell ... - CNET](https://www.cnet.com/tech/theres-ai-and-then-theres-agi-what-you-need-t

## Apify API를 사용한 웹 스크래핑

In [17]:
# Put your Apify API key in the .env file in the same directory as this script
APIFY_API_KEY = os.environ["APIFY_API_KEY"]

In [18]:
from apify_client import ApifyClient
from typing_extensions import Annotated


def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=APIFY_API_KEY)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

In [19]:
from autogen import ConversableAgent, register_function

# Create web scrapper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config=llm_config_dict,
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the function with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

In [20]:
# url = 'https://en.wikipedia.org/wiki/Artificial_general_intelligence'
url = 'https://en.wikipedia.org/wiki/Intelligent_agent'
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message=f"Can you scrape {url} for me?")
    #summary_method="reflection_with_llm")

UserProxy (to WebScraper):

Can you scrape https://en.wikipedia.org/wiki/Intelligent_agent for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_WZkLyPlqUX66QcwiBzzp2FKP): scrape_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Intelligent_agent"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_page...
Can you scrape https://en.wikipedia.org/wiki/Intelligent_agent for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_WZkLyPlqUX66QcwiBzzp2FKP): scrape_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Intelligent_agent"}
*****************************************************

In [21]:
agi_wiki = chat_result.chat_history[2]['content']

In [22]:
agi_wiki

'Intelligent agent - Wikipedia\nIn intelligence and artificial intelligence, an intelligent agent (IA) is an agent that perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge. Agentic AI, a subset of intelligent agents, expands this concept by autonomously pursuing goals, making decisions, and taking actions over extended periods, effectively embodying a novel form of digital agency. \nAn intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.[1] \nSimple reflex agent diagram \nLeading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term bor

In [23]:
with open("agi_wiki_content.txt", "w", encoding="utf-8") as file:
    file.write(agi_wiki)

## 스크랩한 위키 파일에 대해 RAG을 수행

In [24]:
# !pip3 install opentelemetry-exporter-otlp-proto-http

In [25]:
# %pip install "pyautogen[retrievechat]"

In [26]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

DOC_PATH = ["./agi_wiki_content.txt"]

In [27]:
llm_config = {'config_list': llm_config_dict['config_list'], 'timeout': 60, 'cache_seed': 88, 'temperature': 0}

In [28]:
chromadb.PersistentClient(path="/tmp/chromadb")

In [29]:
assistant = AssistantAgent(
    name="assistant",
    human_input_mode="NEVER",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)
        
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "boise_idaho",
        "get_or_create": True, # if True, will create/return a collection for the retrieve chat.
    },
    code_execution_config={"use_docker": False}
)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# prompt = "What is the difference beween AGI and classic AI"
prompt = "What is Intelligent Agent?"
res = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=prompt, n_results=1, silent=True)

2025-02-05 06:41:47,334 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `boise_idaho`.
max_tokens is too small to fit a single line of text. Breaking this line:
	Intelligent agent - Wikipedia ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2025-02-05 06:41:47,428 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 3 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [['b7376d96']]
Adding content of doc b7376d96 to context.


In [31]:
print(f"Question:{prompt}")
print(f"Answer from RAG: {res.chat_history[-1]['content']}")

Question:What is Intelligent Agent?
Answer from RAG: An intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and can improve its performance through learning or acquiring knowledge.


# Group Chat 

지금까지 두 에이전트 또는 두 에이전트 간의 일련의 상호작용을 포함하는 대화 패턴을 알아봤습니다.

AutoGen은 그룹 채팅(**group chat**)이라는 더 광범위한 대화 모델을 소개하며, 이 모델은 두 개 이상의 에이전트를 포함합니다.

그룹 채팅의 기본 개념은 모든 에이전트가 하나의 통합된 대화 스레드에 참여하고 동일한 컨텍스트를 공유한다는 것입니다.

이 접근 방식은 여러 에이전트 간의 협력이 필요한 작업에 유용합니다.


<img src="./images/autogen_image3.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">



그룹 채팅 관리자(Group Chat Manager)는 다음 단계를 통해 그룹 채팅을 오케스트레이션합니다.

1.	그룹 채팅 관리자(Group Chat Manager)가 발언할 에이전트를 선택합니다.
2.	선택된 에이전트가 발언하고, 메시지가 그룹 채팅 관리자에게 다시 전송됩니다.
3.	그룹 채팅 관리자가 메시지를 그룹 내 다른 모든 에이전트에게 전파합니다.
4.	이 과정은 대화가 중단될 때까지 반복됩니다.

그룹 채팅 관리자(Group Chat Manager)는 여러 전략을 사용하여 다음 에이전트를 선택

- **round_robin**:  그룹 채팅 관리자가 제공된 에이전트의 순서에 따라 라운드 로빈 방식으로 에이전트를 선택
- **random**:  그룹 채팅 관리자가 에이전트를 무작위로 선택
- **manual**:  그룹 채팅 관리자가 사람의 입력을 요청하여 에이전트를 선택
- **auto**:  Default 전략으로, 그룹 채팅 관리자의 LLM (대규모 언어 모델)을 사용하여 에이전트를 선택

#### Agents 정의

In [32]:
import os
from autogen import GroupChat, GroupChatManager, UserProxyAgent, AssistantAgent, register_function
from dotenv import load_dotenv
load_dotenv()

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="""A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin. 
    Execute the plan by making sure each agent does their part and help in the report creation.""",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=3
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin.
                    The plan may involve input from Economist, WhiteCollarWorker, and Environmentalist agents.
                    Explain the plan first. Be clear which step is performed by an Economist, 
                    and which step is performed by a WhiteCollarWorker, and and which step is performed Environmentalist.
                    """,
    llm_config=llm_config_dict,
    description="Suggest a plan involving input from other agents."
)

reportagent = AssistantAgent(
    name="Report_Agent",
    system_message="""Once plan is approved, 
                    You are responsible for creating a report by extracting insights from the other agents. 
                    Return report in well formatted markdown format, save the report and TERMINATE.""",
    llm_config=llm_config_dict,
    description="Responsible for creating a report by extracting insights from the chat history."
)

economist = AssistantAgent(
    name="Economist",
    system_message="""You are a highly experienced economist specializing in global economic trends and geopolitical risks. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Expert in global economics and the impact of conflicts."
)

economist = AssistantAgent(
    name="Programmer",
    system_message="""You are a highly experienced software developer specializing in AI and cloud. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Expert in global economics and the impact of conflicts."
)

whitecollarworker = AssistantAgent(
    name="WhiteCollarWorker",
    system_message="""You are an average white collar worker concerned about the economy and the environment. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Highly experienced white collar worker with a pragmatic approach to sustain healthy economy and environment."
)

environmentalist = AssistantAgent(
    name="Environmentalist",
    system_message="""You are an environmentalist advocating for sustainable practices and conservation of natural resources. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
    llm_config=llm_config_dict,
    description="Promotes environmental awareness and sustainable solutions."
)

<img src="./images/autogen_image4.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">


#### 도구로 Agent 기능 강화

<img src="./images/autogen_image5.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

In [33]:
from duckduckgo_search import DDGS
from typing import Annotated
import datetime

def save_report(report: str) -> str:
    try:
        file_name = "report_"+str(datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S"))+ ".md"
        with open(file_name, "w") as f:
            f.write(report)
        return "Report saved successfully."
    except Exception as e:
        print(e)
        return "Failed to save the report."

def browse_web(query: str) -> str:
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=1)]
            return results if results else "Not Found"
    except Exception as e:
        print(e)

def web_search_tool (query: Annotated [str, 'Query string containing information that you want to search using the internet']) -> str:
    result=browse_web(query)
    return result

def save_report_tool(report: Annotated[str, 'Report in markdown format']) -> str:
    result=save_report(report)
    return result

register_function(
    web_search_tool,
    caller=economist,
    executor=economist,
    description="Web Browser Tool to search the internet for information.",
)
register_function(
    web_search_tool,
    caller=whitecollarworker,
    executor=whitecollarworker,
    description="Web Browser Tool to search the internet for information.",
)
register_function(
    web_search_tool,
    caller=environmentalist,
    executor=environmentalist,
    description="Web Browser Tool to search the internet for information.",
)

register_function(
    save_report_tool,
    caller=reportagent,
    executor=reportagent,
    description="Save the report in markdown format.",
)


#### Group Chat Setup

<img src="./images/autogen_image6.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

In [34]:
group_chat = GroupChat(
    agents=[user_proxy, planner, economist, whitecollarworker, environmentalist, reportagent],
    messages=[],
    max_round=25,
    send_introductions=True,
    speaker_selection_method="auto"
)

# Group Chat Manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_dict
)

# Initiate the Conversation
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Create a report on : Analyzing the potential global ramifications of onset of Artificial general intelligence. ",
    # summary_method="reflection_with_llm"
)

User_proxy (to chat_manager):

Create a report on : Analyzing the potential global ramifications of onset of Artificial general intelligence. 



--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Thank you for your request. Although the task of creating a report directly is handled by the Report_Agent, I can devise a plan to analyze the potential global ramifications of the onset of Artificial General Intelligence (AGI) by consulting our respective experts. Here's the outline for the plan:

1. **Initial Assessment by the Programmer:**
   - The Programmer will analyze recent trends and developments in AGI to determine the potential areas where AGI could have global ramifications. This will include sectors like economy, employment, technology, compliance, and societal behavior.

2. **Economic Impact Analysis by the Economist:**
   - The Economist will evaluate potential economic impacts, including shifts in labor markets, changes in productivity, potential GDP growth or contraction, and international economic competitiveness. The analysis should in

In [35]:
# Print the Discussion Summary Report
print(chat_result.summary)

Please refer to https://microsoft.github.io/autogen/docs/ for information.